# EE364 Coin Flipping Example
This is a simple example for a random experiment that we will perform as a class in our first meeting.  It illustrates a number of topics in the class which we will discuss during the semester, including the difference between probability and statistics.  We can return to this as we progress during the semester.

In [ ]:
import numpy as np
import scipy.stats as stats
import matplotlib as mpl
import matplotlib.pyplot as plt

This simulates a class of size `N_students` where each student flips a coin (with heads probability `p`) `N_flips` times.  

In [ ]:
N_students = 10000 # <<---- change this to adjust class size
N_flips = 10
p = 0.75 
np.random.seed(123)  # <<---- change this to get a "new class"

This is a well-known problem in probability, so the results for the theoretical model for this experiment are built into Python.  The following code snippet produces an array `binomial_pmf` with `binomial_pmf[k]` being the probability that `k` heads will be observed when a coin is flipped `N_flips` times.

In [ ]:
heads = np.arange(0, N_flips + 1)
binomial_pmf = stats.binom.pmf(heads, N_flips, p)

Python also has built-in functions to run this experiment `N_students` times -- note that this is different from the theory above since it is simulating the actual random experiments of flipping the coin.

In [ ]:
flip_results = np.random.binomial(N_flips, p, N_students)
flip_counts = np.zeros(N_flips + 1)

## count the number of times k flips were obtained in these N_students experiments
for k in range(N_flips + 1):
    flip_counts[k] = np.sum(flip_results == k)

This code below computes an estimate of `pmf_hat[k]`, the probability of getting `k` heads out of `N_flips` flips, as the relative frequency of occurance of heads during the experiments.  It also computes a "Margin of Error" for our estimate, which is a measure of our statistcial confidence in this estimate. 

In [ ]:
pmf_hat = flip_counts / N_students
MOE = np.zeros((2, N_flips + 1))  ## upper and lower error range for each 
MOE[0] = 1.96 * np.sqrt( pmf_hat * (1.0 - pmf_hat) / N_students  )
 
print(f'number of students = {N_students}')
print(f'number of flips =  {N_flips}')
print(f'probability of heads = {p}')
print('\nk\tpmf(k)\t\tn(k)\tpmf_hat(k)\tMOE(k)')

for k in range(len(pmf_hat)):
    if flip_counts[k] == 0 : 
        MOE[1][k] = 3 / N_students
    else:
        MOE[1][k] = MOE[0][k]
    print(f'{k}:\t{binomial_pmf[k] : 0.4e}\t{flip_counts[k] : 4.0f}\t{pmf_hat[k] : 0.4e}\t{MOE[1][k] : 0.4g}')

MOE[0] = np.clip(MOE[0], 0, pmf_hat )  # limit confidence region to be > 0 since we know probabilities are >=0


Finally, let's plot the theoretical model for the probability of getting `k` heads in `N_flips` and compare that to the fraction of students who observed `k` heads during their experiment.

In [ ]:
# swiped from: https://matplotlib.org/examples/api/barchart_demo.html
width = 0.35 
mpl.rcParams["errorbar.capsize"] = 4     
fig, ax = plt.subplots(figsize=(10,7))
rects1 = ax.bar(heads, binomial_pmf, width)
rects2 = ax.bar(heads+width, pmf_hat, width, yerr=MOE)
ax.set_xticks(heads + width / 2)
ax.set_xticklabels(heads)
ax.set_title('Results of Coin Flipping Experiment')
ax.set_xlabel(f'number of heads in {N_flips} flips')
ax.set_ylabel(f'probability or relative frequency')
ax.legend((rects1[0], rects2[0]), ('Model', 'Experimental'))